In [1]:
from datetime import datetime as dt
import re
import pandas as pd

In [2]:
with open("data/carlsen.pgn", "r") as f:
    pgn = f.read()

In [3]:
chunks = pgn.split('\n\n')

In [4]:
info = chunks[::2][:-1]

In [5]:
info[0]

'\ufeff[Event "FTX Crypto Cup KO"]\n[Site "chess24.com INT"]\n[Date "2021.05.31"]\n[Round "3.33"]\n[White "Carlsen, Magnus"]\n[Black "So, Wesley"]\n[Result "1-0"]\n[ECO "C54"]\n[WhiteElo "2847"]\n[BlackElo "2770"]\n[PlyCount "87"]\n[EventDate "2021.05.26"]\n[EventType "k.o. (rapid)"]\n[EventRounds "3"]\n[EventCountry "GER"]\n[SourceTitle "CB22_2021"]\n[SourceDate "2021.06.02"]\n[SourceVersion "1"]\n[SourceVersionDate "2021.06.02"]\n[SourceQuality "1"]'

In [6]:
name = "Carlsen, Magnus"

In [7]:
def get_tag(tag, info):
    s = rf'{tag} \"(.+?)\"'
    matches = re.search(s, info)
    return matches.groups()[0] if matches else None

In [8]:
events, rounds, results, opponent_elos, dates = [],[],[],[],[]

for game in info:
    events.append(get_tag("Event", game))
    rounds.append(get_tag("Round", game))
    
    white = get_tag("White", game)
    black = get_tag("Black", game)

    result = get_tag("Result", game)

    if white == name:
        i = 0
        opponent_elo = get_tag("BlackElo", game)
    elif black == name:
        i = 1
        opponent_elo = get_tag("WhiteElo", game)
    else:
        raise Exception("No matching player!")
    
    opponent_elos.append(opponent_elo)

    point_str = result.split('-')[i]
    if point_str == "0":
        points = 0
    elif point_str == "1":
        points = 1
    elif point_str == "1/2":
        points = 0.5
    
    results.append(points)
    
    dates.append(dt.strptime(get_tag("Date", game), "%Y.%m.%d"))

In [9]:
df = pd.DataFrame({
    "Event": events,
    "Round": rounds,
    "Result": results,
    "OpponentELO": opponent_elos,
    "Date": dates,
})

In [10]:
df.shape

(1000, 5)

In [11]:
df.head()

,Event,Round,Result,OpponentELO,Date
0,FTX Crypto Cup KO,3.33,1.0,2770,2021-05-31
1,FTX Crypto Cup KO,3.32,1.0,2770,2021-05-31
2,FTX Crypto Cup KO,3.31,0.0,2770,2021-05-31
3,FTX Crypto Cup KO,3.24,0.5,2770,2021-05-31
4,FTX Crypto Cup KO,3.23,0.5,2770,2021-05-31


In [12]:
df.tail()

,Event,Round,Result,OpponentELO,Date
995,Sinquefield Cup playoff2 7th,3,0.0,2805,2019-08-29
996,Sinquefield Cup playoff1 7th,2,0.5,2805,2019-08-29
997,Sinquefield Cup playoff1 7th,1,0.5,2805,2019-08-29
998,Sinquefield Cup 7th,11,1.0,2778,2019-08-28
999,Sinquefield Cup 7th,10,1.0,2776,2019-08-27


In [43]:
df['Date'].max()

Timestamp('2021-05-31 00:00:00')

In [44]:
df['Date'].min()

Timestamp('2019-08-28 00:00:00')

In [15]:
df = df[df['Round'] != "?"] # remove events without round information

In [16]:
df.shape

(982, 5)

The Chessbase database has some rounds in a format like 3.33, etc. To get the proper round order I split the round column into two parts.

In [17]:
df['r1'] = df['Round'].apply(lambda r: int(r.split('.')[0]))
df['r2'] = df['Round'].apply(lambda r: int(r.split('.')[-1]) if '.' in r else 0)
df = df.sort_values(["Event", "r1", "r2"]).reset_index(drop=True)

In [18]:
df.head()

,Event,Round,Result,OpponentELO,Date,r1,r2
0,Airthings Masters KO,1.11,1.0,2702,2020-12-29,1,11
1,Airthings Masters KO,1.12,0.5,2702,2020-12-29,1,12
2,Airthings Masters KO,1.13,0.0,2702,2020-12-29,1,13
3,Airthings Masters KO,1.14,0.5,2702,2020-12-29,1,14
4,Airthings Masters KO,1.21,0.0,2702,2020-12-30,1,21


In [19]:
df.tail()

,Event,Round,Result,OpponentELO,Date,r1,r2
977,Wch Rapid,13,1.0,2772,2019-12-28,13,0
978,Wch Rapid,14,0.5,2758,2019-12-28,14,0
979,Wch Rapid,15,0.5,2736,2019-12-28,15,0
980,World Blitz 2019,12.2,1.0,2731,2019-12-29,12,2
981,World Rapid 2019,12.1,1.0,2775,2019-12-28,12,1


In [20]:
df.to_csv("tmp.csv", index=False)

In [21]:
# remove bullet and simuls
df['Event'] = df['Event'].str.lower()
df = df[~df["Event"].str.contains('bullet')]
df = df[~df["Event"].str.contains('simul')]

In [22]:
df.shape

(808, 7)

In [23]:
df['OpponentELO'] = df['OpponentELO'].astype('int')

In [24]:
df['previous'] = df['Result'].shift()

In [25]:
df.head()

,Event,Round,Result,OpponentELO,Date,r1,r2,previous
0,airthings masters ko,1.11,1.0,2702,2020-12-29,1,11,NaN
1,airthings masters ko,1.12,0.5,2702,2020-12-29,1,12,1.0
2,airthings masters ko,1.13,0.0,2702,2020-12-29,1,13,0.5
3,airthings masters ko,1.14,0.5,2702,2020-12-29,1,14,0.0
4,airthings masters ko,1.21,0.0,2702,2020-12-30,1,21,0.5


In [26]:
df = df.reset_index(drop=True)

In [27]:
df.to_csv("tmp.csv")

Now that the previous round data has been computed, remove the first round from each event.

In [28]:
first_rounds = df.reset_index().groupby('Event').first()['index'].values
df = df.drop(index=first_rounds)

In [29]:
df.shape

(763, 8)

In [30]:
df.to_csv("tmp.csv", index=False)

In [31]:
def perf_rating(p, rc): # https://en.wikipedia.org/wiki/Performance_rating_(chess) - linear method
    dp = (800 * p) - 400
    pr = rc + dp
    return pr

In [32]:
win = df[df['previous'] == 1]
loss = df[df['previous'] == 0]
draw = df[df['previous'] == 0.5]

In [33]:
len(win), len(loss), len(draw)

(336, 116, 311)

In [34]:
def get_stats(df):
    p = df['Result'].mean()
    rc = df['OpponentELO'].mean()
    pr = perf_rating(p, rc)
    return p, pr

In [35]:
score_df = pd.DataFrame(columns = ['Win', 'Loss', 'Draw'])
perf_df = pd.DataFrame(columns = ['Win', 'Loss', 'Draw'])

In [36]:
# perf_df = pd.read_csv("perf.csv", index_col=0)
# score_df = pd.read_csv("score.csv", index_col=0)

In [37]:
score_df.loc[name, 'Win'], perf_df.loc[name, 'Win'] = get_stats(win)
score_df.loc[name, 'Draw'], perf_df.loc[name, 'Draw'] = get_stats(draw)
score_df.loc[name, 'Loss'], perf_df.loc[name, 'Loss'] = get_stats(loss)

In [38]:
score_df = score_df.astype("float").round(2)

In [39]:
perf_df = perf_df.round().astype("int")

In [40]:
score_df

,Win,Loss,Draw
"Carlsen, Magnus",0.65,0.72,0.61


In [41]:
perf_df

,Win,Loss,Draw
"Carlsen, Magnus",2854,2920,2830


In [42]:
# score_df.to_csv("score.csv")
# perf_df.to_csv("perf.csv")